In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import ipywidgets as widgets
from IPython.display import display, clear_output

In [20]:
# Load the dataset with a different encoding
df = pd.read_csv('Superstore.csv', encoding='ISO-8859-1')

In [21]:
df_clean = df.copy()

In [22]:
cols_to_drop = ['Row ID', 'Order ID', 'Customer ID', 'Customer Name',
                'Product ID', 'Product Name', 'Country', 'Postal Code']

df_clean.drop(columns=cols_to_drop, inplace=True)

# Check for missing values
missing_values = df_clean.isnull().sum()
print("\nMissing Values:")
display(missing_values)


Missing Values:


,0
Order Date,0
Ship Date,0
Ship Mode,0
Segment,0
City,0
State,0
Region,0
Category,0
Sub-Category,0
Sales,0


In [23]:
# Convert date columns to datetime
df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'])
df_clean['Ship Date'] = pd.to_datetime(df_clean['Ship Date'])

# Create new features
df_clean['Order_Month'] = df_clean['Order Date'].dt.month
df_clean['Shipping_Duration'] = (df_clean['Ship Date'] - df_clean['Order Date']).dt.days

# Drop the original date columns
df_clean.drop(columns=['Order Date', 'Ship Date'], inplace=True)

# Preview the updated dataset
print("\nProcessed Dataset:")
display(df_clean.head())


Processed Dataset:


,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit,Order_Month,Shipping_Duration
0,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,261.9600,2,0.00,41.9136,11,3
1,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,731.9400,3,0.00,219.5820,11,3
2,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,14.6200,2,0.00,6.8714,6,4
3,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775,5,0.45,-383.0310,10,7
4,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680,2,0.20,2.5164,10,7


In [24]:
# One-hot encode categorical features
df_encoded = pd.get_dummies(df_clean, drop_first=True)

# Preview the encoded dataset
print("\nEncoded Dataset (first few columns):")
display(df_encoded.iloc[:, :10].head())


Encoded Dataset (first few columns):


,Sales,Quantity,Discount,Profit,Order_Month,Shipping_Duration,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class,Segment_Corporate
0,261.9600,2,0.00,41.9136,11,3,False,True,False,False
1,731.9400,3,0.00,219.5820,11,3,False,True,False,False
2,14.6200,2,0.00,6.8714,6,4,False,True,False,True
3,957.5775,5,0.45,-383.0310,10,7,False,False,True,False
4,22.3680,2,0.20,2.5164,10,7,False,False,True,False


In [25]:
# Split features and target variable
X = df_encoded.drop(columns=['Profit'])
y = df_encoded['Profit']

# Split the data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the shape of the splits
print(f"\nTraining set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")


Training set size: (7995, 609)
Test set size: (1999, 609)


In [26]:
# Initialize models
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42, verbosity=0)

# Dictionary to store results
results = {}

# Helper function to train and evaluate
def evaluate_model(name, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = {"R2": r2, "MAE": mae, "RMSE": rmse}
    print(f"{name} Results:\n R²: {r2:.4f} | MAE: ${mae:.2f} | RMSE: ${rmse:.2f}\n")

In [27]:
# Evaluate each model
print("\n===== Model Training and Evaluation =====")
evaluate_model("Linear Regression", lr)
evaluate_model("Random Forest", rf)
evaluate_model("XGBoost", xgb)


===== Model Training and Evaluation =====
Linear Regression Results:
 R²: -0.6317 | MAE: $72.57 | RMSE: $281.27

Random Forest Results:
 R²: -0.0643 | MAE: $26.95 | RMSE: $227.16

XGBoost Results:
 R²: 0.8143 | MAE: $22.14 | RMSE: $94.88



In [28]:
# View all results
print("📊 Model Comparison Summary:")
for model, scores in results.items():
    print(f"{model}: R² = {scores['R2']:.4f}, MAE = ${scores['MAE']:.2f}, RMSE = ${scores['RMSE']:.2f}")

# Save reference to categorical columns for prediction interface
categorical_columns = {
    'Segment': df_clean['Segment'].unique().tolist(),
    'Ship Mode': df_clean['Ship Mode'].unique().tolist(),
    'Category': df_clean['Category'].unique().tolist(),
    'Sub-Category': df_clean['Sub-Category'].unique().tolist(),
    'Region': df_clean['Region'].unique().tolist(),
    'State': df_clean['State'].unique().tolist()
}

📊 Model Comparison Summary:
Linear Regression: R² = -0.6317, MAE = $72.57, RMSE = $281.27
Random Forest: R² = -0.0643, MAE = $26.95, RMSE = $227.16
XGBoost: R² = 0.8143, MAE = $22.14, RMSE = $94.88


In [29]:
# Find reference categories (those that were dropped during one-hot encoding)
reference_categories = {}
for col, values in categorical_columns.items():
    for val in values:
        col_name = f"{col}_{val}"
        if col_name not in X.columns:
            reference_categories[col] = val
            break

print("\nReference categories (dropped during one-hot encoding):")
for col, val in reference_categories.items():
    print(f"{col}: {val}")


Reference categories (dropped during one-hot encoding):
Segment: Consumer
Ship Mode: First Class
Category: Furniture
Sub-Category: Accessories
Region: Central
State: Alabama


In [30]:
# === Create the prediction interface ===
# Input widgets for numerical features
sales_input = widgets.FloatText(value=100.0, description='Sales:')
quantity_input = widgets.IntText(value=1, description='Quantity:')
discount_input = widgets.FloatText(value=0.0, description='Discount:')
order_month_input = widgets.IntSlider(value=6, min=1, max=12, description='Order Month:')
shipping_duration_input = widgets.IntSlider(value=3, min=0, max=10, description='Ship Days:')

# Dropdown for main categorical features
segment_input = widgets.Dropdown(
    options=categorical_columns['Segment'],
    value=reference_categories.get('Segment', categorical_columns['Segment'][0]),
    description='Segment:'
)

ship_mode_input = widgets.Dropdown(
    options=categorical_columns['Ship Mode'],
    value=reference_categories.get('Ship Mode', categorical_columns['Ship Mode'][0]),
    description='Ship Mode:'
)

category_input = widgets.Dropdown(
    options=categorical_columns['Category'],
    value=reference_categories.get('Category', categorical_columns['Category'][0]),
    description='Category:'
)

sub_category_input = widgets.Dropdown(
    options=categorical_columns['Sub-Category'],
    value=reference_categories.get('Sub-Category', categorical_columns['Sub-Category'][0]),
    description='Sub-Category:'
)

region_input = widgets.Dropdown(
    options=categorical_columns['Region'],
    value=reference_categories.get('Region', categorical_columns['Region'][0]),
    description='Region:'
)

state_input = widgets.Dropdown(
    options=categorical_columns['State'],
    value=reference_categories.get('State', categorical_columns['State'][0]),
    description='State:'
)

# City input - we'll simplify by using a text field since there are too many cities
city_input = widgets.Text(value='New York City', description='City:')

predict_button = widgets.Button(description="Predict Profit", button_style='success')
output = widgets.Output()

In [31]:
# === Predict function ===
def on_predict_clicked(b):
    with output:
        clear_output(wait=True)

        # Create a dataframe with all columns from X, initialized to zeros
        model_input = pd.DataFrame(0, index=[0], columns=X.columns)

        # Fill in the numeric values
        model_input['Sales'] = sales_input.value
        model_input['Quantity'] = quantity_input.value
        model_input['Discount'] = discount_input.value
        model_input['Order_Month'] = order_month_input.value
        model_input['Shipping_Duration'] = shipping_duration_input.value

        # Set the appropriate categorical columns based on selections
        # Only set to 1 if it's not the reference category (which was dropped during one-hot encoding)
        if segment_input.value != reference_categories.get('Segment'):
            col = f'Segment_{segment_input.value}'
            if col in model_input.columns:
                model_input[col] = 1

        if ship_mode_input.value != reference_categories.get('Ship Mode'):
            col = f'Ship Mode_{ship_mode_input.value}'
            if col in model_input.columns:
                model_input[col] = 1

        if category_input.value != reference_categories.get('Category'):
            col = f'Category_{category_input.value}'
            if col in model_input.columns:
                model_input[col] = 1

        if sub_category_input.value != reference_categories.get('Sub-Category'):
            col = f'Sub-Category_{sub_category_input.value}'
            if col in model_input.columns:
                model_input[col] = 1

        if region_input.value != reference_categories.get('Region'):
            col = f'Region_{region_input.value}'
            if col in model_input.columns:
                model_input[col] = 1

        if state_input.value != reference_categories.get('State'):
            col = f'State_{state_input.value}'
            if col in model_input.columns:
                model_input[col] = 1

        # For city, we won't try to set it directly as there are too many
        # In a real application, you'd want to implement this more robustly
        city_col = f'City_{city_input.value}'
        if city_col in model_input.columns:
            model_input[city_col] = 1

        # Make prediction
        prediction = xgb.predict(model_input)[0]
        margin = prediction / sales_input.value * 100 if sales_input.value > 0 else 0

        print(f"💰 Predicted Profit: ${prediction:.2f}")
        print(f"📊 Profit Margin: {margin:.2f}%")

        # Provide insights
        if prediction < 0:
            print("⚠️ Warning: This transaction is predicted to lose money!")
        elif margin < 10 and sales_input.value > 0:
            print("⚠️ Note: The profit margin is below 10%.")

        # Get feature importance for this prediction
        if hasattr(xgb, 'feature_importances_'):
            # Sort features by importance for this specific prediction
            importances = xgb.feature_importances_
            feature_importance = pd.DataFrame({
                'Feature': X.columns,
                'Importance': importances
            }).sort_values('Importance', ascending=False).head(5)

            print("\n📈 Top 5 factors influencing this prediction:")
            for i, row in feature_importance.iterrows():
                print(f"- {row['Feature']}: {row['Importance']:.4f}")

In [32]:
# Set up the button click handler
predict_button.on_click(on_predict_clicked)

# Organize widgets in tabs for better layout
tab1 = widgets.VBox([
    widgets.HBox([sales_input, quantity_input, discount_input]),
    widgets.HBox([order_month_input, shipping_duration_input])
])

tab2 = widgets.VBox([
    widgets.HBox([segment_input, ship_mode_input]),
    widgets.HBox([category_input, sub_category_input])
])

tab3 = widgets.VBox([
    widgets.HBox([region_input, state_input]),
    widgets.HBox([city_input])
])

tabs = widgets.Tab(children=[tab1, tab2, tab3])
tabs.set_title(0, 'Basic Info')
tabs.set_title(1, 'Product Info')
tabs.set_title(2, 'Location')

In [33]:
# Display the interface
print("\n===== Profit Prediction Interface =====")
print("Use the tabs below to input transaction details and predict profit")
display(widgets.VBox([
    tabs,
    predict_button,
    output
]))


===== Profit Prediction Interface =====
Use the tabs below to input transaction details and predict profit


In [34]:
# Feature importance analysis for the model
if hasattr(xgb, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': xgb.feature_importances_
    }).sort_values('Importance', ascending=False).head(15)

    print("\n===== Model Insights =====")
    print("Top 15 features influencing profit in the XGBoost model:")
    display(feature_importance)


===== Model Insights =====
Top 15 features influencing profit in the XGBoost model:


,Feature,Importance
598,Sub-Category_Copiers,0.367391
2,Discount,0.144984
595,Sub-Category_Binders,0.128898
592,Category_Technology,0.043788
0,Sales,0.036135
589,Region_South,0.028418
588,Region_East,0.028131
608,Sub-Category_Tables,0.017824
591,Category_Office Supplies,0.014592
607,Sub-Category_Supplies,0.011976
